## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 3.1. SceneBuilder

##### Create CombinedRobot instance - you need it to initailize GeometryScene

In [2]:
from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.0.63"
PANDA_HOST_IP = "192.168.0.172" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.0.13" ## The robot has it's own IP

combined_robot = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))], 
    connection_list=[False, False])

connection command:
indy0: False
panda1: False


##### initialize ArucoStereo detector

In [3]:
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

# stereo = ArucoStereo(aruco_map=get_aruco_map(), 
#                      camera_list=[Kinect(), RealSense()])

# stereo.initialize()

# time.sleep(1) # Let the camera have some time to get stable
# stereo.calibrate()
# print("Detector initialized")

##### Create SceneBuilder instance

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
scene_builder = SceneBuilder.instance(None)
# scene_builder = SceneBuilder.instance(detector=stereo)

##### reset_reference_coord()
* Set the name of coordinate reference object - here, you need "floor" as reference

In [5]:
# scene_builder.reset_reference_coord(ref_name="floor")

##### detect_items()
* Detect items in the field of view. Here, we detect robot positions and update it to the combined robot

In [6]:
# xyz_rpy_robots = scene_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# combined_robot.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)

##### create_gscene()
* Create a GeometryScene instance

In [7]:
gscene = scene_builder.create_gscene(combined_robot)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


##### detect_and_register()
* Detect items in the field of view and register them to the GeometryScene
* They will appear in the RVIZ

In [8]:
# gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
# gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

## FT sensor

In [9]:
import sys
RNB_TRACK_DIR = os.path.join(os.environ['RNB_TRACK_DIR'])
sys.path.append(RNB_TRACK_DIR)
from robotous_ft import *
rft = RobotousFT()

In [15]:
# gscene.show_pose(combined_robot.home_pose)
Tbe = gscene.get_tf("indy0_tcp", )
for _ in range(100):
    ft = rft.read_once()
    gscene.show_wrench("ft", ft*10, Tbe)
    time.sleep(0.05)

## Vive
#### Prepare VR
1. Start vrmonitor
2. Turn on trackers - push until led blinks or change to green - Trackers will be displayed green on the SteamVR
  - Turn on the reference tracker first

In [9]:
from pkg.utils.shared_function import clear_channels_on
from pkg.detector.vive.shared_vive import SharedTrackers, SERVER_ID

In [10]:
# clear_channels_on(SERVER_ID)
track = SharedTrackers()
track.init_vive()

#### Set reference

In [11]:
vive_ref = gscene.NAME_DICT["vive_ref"]
tf_base_ref = vive_ref.get_tf(combined_robot.home_pose)
track.set_reference(ref_name="tracker_1", tf_base_ref=tf_base_ref)

#### show tracker coordinates

In [12]:
for _ in range(50):
    try:
        pose_dict = track.get_all_pose()
        for tname, T in pose_dict.items():
            gscene.add_highlight_axis("hl", tname, T=np.array(T))
        time.sleep(0.1)
    except Exception as e:
        print(e)

#### Draw error arrow

In [13]:
from pkg.utils.rotation_utils import *
from pkg.utils.plot_utils import ArrowStream, draw_background_multi

In [14]:
ref_pos = track.get_pose(tname="tracker_2")
xy_ref = ref_pos[:2, 3]
theta_ref = Rot2axis(ref_pos[:3, :3], 3)
gscene.add_highlight_axis("hl", "tracker_2_ref", T=ref_pos)
def arrow_update_xy():
    try:
        cur_pos = track.get_pose(tname="tracker_2")
        gscene.add_highlight_axis("hl", "tracker_2", T=cur_pos)
        arrow_update_xy.cur_pos = cur_pos
    except:
        cur_pos = arrow_update_xy.cur_pos
    xy_cur = cur_pos[:2, 3]
    theta_cur = Rot2axis(cur_pos[:3, :3], 3)
    
    xy_diff = np.subtract(xy_ref, xy_cur)
    theta_diff = np.subtract(theta_ref, theta_cur)
    
    length = np.linalg.norm(xy_diff)
    
    return length, np.arctan2(xy_diff[1], xy_diff[0]), (0,0,1)

arrow_update_xy()

def arrow_update_theta():
    cur_pos = arrow_update_xy.cur_pos
    xy_cur = cur_pos[:2, 3]
    theta_cur = Rot2axis(cur_pos[:3, :3], 3)
    
    xy_diff = np.subtract(xy_ref, xy_cur)
    theta_diff = np.subtract(theta_ref, theta_cur)
    theta_diff = (theta_diff + np.pi) % (np.pi*2)-np.pi
    
    if theta_diff >= 0:
        angle=0
    else:
        angle=np.pi
    
    return np.abs(theta_diff)/np.pi, angle, (1,0,0)

In [25]:
astream_xy = ArrowStream(arrow_update_xy, sname="xy")
astream_theta = ArrowStream(arrow_update_theta, im_size=(200, 500), sname="theta")

In [26]:
draw_background_multi([astream_xy, astream_theta])

In [23]:
astream_xy.stop_now()
astream_theta.stop_now()